# Météo et hotels

In [1]:
import requests as r
import json
import config
import pandas as pd

# 35 villes de France à visiter

KEY=config.api_key
city_list = [
"Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

# get coordinates of the cities

base_endpoint = 'https://nominatim.openstreetmap.org/search?'

coordinate_list = []

for i, city in enumerate(city_list):
    params = {'q': city, 'format': 'json'}
    response = r.get(base_endpoint, params= params)

    for search_result in response.json():
        if 'france' in search_result['display_name'].lower():
            coordinate_list.append((city, search_result['lat'],search_result['lon']))
            print(f'{i+1} requêtes faites')
            break

1 requêtes faites
2 requêtes faites
3 requêtes faites
4 requêtes faites
5 requêtes faites
6 requêtes faites
7 requêtes faites
8 requêtes faites
9 requêtes faites
10 requêtes faites
11 requêtes faites
12 requêtes faites
13 requêtes faites
14 requêtes faites
15 requêtes faites
16 requêtes faites
17 requêtes faites
18 requêtes faites
19 requêtes faites
20 requêtes faites
21 requêtes faites
22 requêtes faites
23 requêtes faites
24 requêtes faites
25 requêtes faites
26 requêtes faites
27 requêtes faites
28 requêtes faites
29 requêtes faites
30 requêtes faites
31 requêtes faites
32 requêtes faites
33 requêtes faites
34 requêtes faites
35 requêtes faites


## Météo API

In [5]:
# prévisions à 4 jours

base_endpoint = 'https://api.openweathermap.org/data/2.5/forecast?'
KEY = config.api_key

def cleaning_dict(forecast_dict):

    clean_dict = {}
    
    for key in forecast_dict.keys():
        if type(forecast_dict[key]) == list:

            for i in forecast_dict[key][0].keys():
                clean_dict[key+'_'+i] = forecast_dict[key][0][i]

        elif type(forecast_dict[key]) == dict:

            for i in forecast_dict[key].keys():
                clean_dict[key+'_'+i] = forecast_dict[key][i]

        else:
            clean_dict[key] = forecast_dict[key]

    return clean_dict

In [6]:
def one_city_request(base_endpoint, KEY, city_loc):
    
    params = {'lat': city_loc[1],
          'lon': city_loc[2],
          'appid': KEY,
          'units': 'metric'
          }

    response = r.get(base_endpoint, params=params)
    print(response.json)
    return response.json()['list']

In [7]:
result_list = []
for i, city_loc in enumerate(coordinate_list):
    
    weather_request = one_city_request(base_endpoint, KEY, city_loc)

    for forecast_dict in weather_request:

        base_dict = {'id': i,
                'city': city_loc[0],
                'lat': city_loc[1],
                'lon': city_loc[2],
                }
        
        base_dict.update(cleaning_dict(forecast_dict))
        result_list.append(base_dict)

    print(f'{i+1} villes traitées')

<bound method Response.json of <Response [200]>>
1 villes traitées
<bound method Response.json of <Response [200]>>
2 villes traitées
<bound method Response.json of <Response [200]>>
3 villes traitées
<bound method Response.json of <Response [200]>>
4 villes traitées
<bound method Response.json of <Response [200]>>
5 villes traitées
<bound method Response.json of <Response [200]>>
6 villes traitées
<bound method Response.json of <Response [200]>>
7 villes traitées
<bound method Response.json of <Response [200]>>
8 villes traitées
<bound method Response.json of <Response [200]>>
9 villes traitées
<bound method Response.json of <Response [200]>>
10 villes traitées
<bound method Response.json of <Response [200]>>
11 villes traitées
<bound method Response.json of <Response [200]>>
12 villes traitées
<bound method Response.json of <Response [200]>>
13 villes traitées
<bound method Response.json of <Response [200]>>
14 villes traitées
<bound method Response.json of <Response [200]>>
15 ville

In [8]:
data_weather = pd.DataFrame(result_list)
data_weather.head()
col_a_garder=["city","lat","lon","weather_main","dt_txt"]

df_selected = data_weather[col_a_garder]

derniers_enregistrements = df_selected.groupby('city').last().reset_index()

derniers_enregistrements.to_csv('Temps4Mars2024.csv')

## Hotels scrapping Booking.com

In [9]:
from bs4 import BeautifulSoup

In [10]:
agent = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.6261.95 Safari/537.36'}
search_urls = []
for i in range(len(derniers_enregistrements)):
    search_urls.append('https://www.booking.com/searchresults.fr.html?ss='+derniers_enregistrements['city'][i]+'&nflt=review_score%3D60')

hotels_urls = []
for url in search_urls:
    page = r.get(url, headers=agent)
    soup = BeautifulSoup(page.content, "html.parser")
    for a in soup.find_all('a',{"class": "a78ca197d0"}, href=True):
        hotels_urls.append(a['href'])

hotels_urls = [i.split('?')[0] for i in hotels_urls] 


In [11]:
class BookingHotelInfo:
    def __init__(self):
        self.hotel_list = []
        self.hotel_data = {}
        self.hotel_data["name"] = "Empty"
        self.hotel_data["address"] = "Empty"
        self.hotel_data["description"] = "Empty"
        self.hotel_data["score"] = "Empty"
        self.hotel_data["url"] = "Empty"
        self.hotel_data["bonus"] = "Empty"
        self.hotel_data["latlon"] = "Empty"

    def get_hotel_data(self, url):
        agent = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
        session_object = r.Session()
        page = session_object.get(url,headers=agent)
        soup = BeautifulSoup(page.content, "lxml")
        try:
            self.hotel_data["name"] = soup.find("h2", {"class": "pp-header__title"}).text.strip()
        except:
            pass
        try:
            self.hotel_data["address"] = soup.find("span", {"class": "hp_address_subtitle"}).text.strip()
        except:
            pass
        try:
            self.hotel_data["description"] = soup.find("div", {"class": "hp_desc_main_content"}).text.strip()
        except:
            pass
        try:
            self.hotel_data["score"] = soup.find("a", {"class": "hp_review_score"}).text.strip()
        except:
            pass
        try:
             self.hotel_data["bonus"] = soup.find("div", {"class": "hp_desc_important_facilities"}).text.strip()
        except:
            pass
        try:
             self.hotel_data["latlon"] = soup.find("a", {"class": "show_map"}).get('data-atlas-latlng')
        except:
            pass
        self.hotel_data["url"] = url
# Try/pass to avoid getting stuck in case there are changes to the site
        self.hotel_list.append(self.hotel_data)


hotels_list = []
for url in hotels_urls:
    hotels = BookingHotelInfo()
    hotels.get_hotel_data(url)
    hotels_list.append(hotels.hotel_list[:])

# Getting hotel data for each hotel on each page

In [13]:
hotels_list

[[{'name': 'La Villa Mazarin',
   'address': '35 boulevard Gambetta, 30220 Aigues-Mortes, France',
   'description': "Bénéficiez d'un traitement VIP grâce au service exclusif de l'établissement La Villa Mazarin\n\n\n\nVous pouvez bénéficier d'une réduction Genius dans l'établissement La Villa Mazarin\xa0! Connectez-vous pour économiser.\nLa Villa Mazarin est construite dans un bâtiment datant du XVème siècle situé dans la ville fortifiée d'Aigues-Mortes et propose une terrasse ainsi qu'un bar. Vous bénéficierez gracieusement d'une connexion Wi-Fi disponible dans l'ensemble de l'établissement.\n\nLes chambres climatisées possèdent le chauffage, une télévision par satellite, un téléphone, un réfrigérateur et du mobilier de style provençal du XVIIème siècle.\n\nLe Salon Mazarin est le restaurant de l'hôtel où vous dégusterez des spécialités culinaires françaises ou une sélection de repas légers et de collations. Le salon s'ouvre sur la cour et sur une terrasse bien exposée.\n\nMontpellier

In [17]:
df_sov = pd.DataFrame(hotels_list, columns=['Colonne'])

# Sauvegarder le DataFrame au format CSV
df_sov.to_csv('ma_liste.csv', index=False)

In [18]:
import boto3

In [33]:
session = boto3.Session(profile_name='default')
s3 = session.resource("s3")
s3_client = session.client("s3")
bucket_name = 'ys42jedha'
bucket = s3.Bucket(bucket_name)

bucket.upload_file('hotels.csv', 'hotels.csv')
bucket.upload_file('Temps4Mars2024.csv', 'Temps4Mars2024.csv')


In [29]:
derniers_enregistrements.columns

Index(['city', 'lat', 'lon', 'weather_main', 'dt_txt'], dtype='object')

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(data_weather, lat="lat", lon="lon", hover_name = 'city', zoom = 4.5,
                        hover_data={
        'lat': False,
        'lon': False,
        }, 
                        color = 'weather_main', color_continuous_scale = px.colors.sequential.Rainbow, size='main_temp',
                        mapbox_style="carto-positron",width = 1200, height = 800,
                        title='Cities with best weather in France over the next 7 days')
fig.show()

In [41]:
data_weather.columns

Index(['id', 'city', 'lat', 'lon', 'dt', 'main_temp', 'main_feels_like',
       'main_temp_min', 'main_temp_max', 'main_pressure', 'main_sea_level',
       'main_grnd_level', 'main_humidity', 'main_temp_kf', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon', 'clouds_all',
       'wind_speed', 'wind_deg', 'wind_gust', 'visibility', 'pop', 'rain_3h',
       'sys_pod', 'dt_txt', 'snow_3h'],
      dtype='object')

In [ ]:
data_weather['lat'] = data_weather['lat'].astype(float)
data_weather['lon'] = data_weather['lon'].astype(float)
fig = px.scatter_mapbox(data_weather, lat="lat", lon="lon", hover_name="city",
                        hover_data=
                        ["weather_main"], zoom=3, height=600)

# Personnalisez le style de la carte Mapbox
fig.update_layout(mapbox_style="open-street-map")

# Affiche la carte
fig.show()